In [223]:
import numpy as np, time, csv
import os,sys, csv, pickle

In [224]:
import cv2
from collections import Counter, OrderedDict, defaultdict

In [225]:
#path = r'D:\code\rough'
#path = r'D:\cogni\MSA'
#os.chdir(path)
path=%pwd


In [226]:
#filename, ext = 'Cognizant CW2398342 2017 Amend2 rate card and Fx update FINAL doc_page_no_6' , '.jpg'
filename, ext = 'sample5' , '.jpg'
orig_img = cv2.imread(''.join((filename, ext)),0)


In [227]:
#orig_img = cv2.imread('sample_loan01.jpg',0)
ret, thresh = cv2.threshold(orig_img.copy(), 220, 255 , 0)
img = thresh.copy()
img2 = img.copy()
cv2.imwrite('thresh.jpg', img)

True

In [228]:
#img[200,-5:-1], img[200,]
orig_img[100:103, 28:38]

array([[220, 220, 220, 221, 222, 221, 221, 221, 221, 220],
       [221, 222, 222, 222, 223, 222, 221, 222, 222, 222],
       [222, 223, 224, 224, 223, 221, 220, 220, 221, 222]], dtype=uint8)

In [229]:
print(np.count_nonzero(img) / float(img.shape[0] *img.shape[1]) )

0.9264581738864228


In [230]:
def font_is_dark(img):
    h, w = img.shape
    
    f_count = Counter(img[0])
    l_count = Counter(img[h-1])
    # if number of white pixels >= 50% then it's a white background with dark font
    if np.count_nonzero(img) / float(h * w) >= 0.5:
        dark = False
    else:
        dark = True

    return dark
    

In [231]:
if font_is_dark(thresh):
    val = 255
else:
    val = 0
val

0

In [232]:
arr1 = np.zeros((15,1))
arr1[0:10,0] = 10
#arr1[10:15,0] = 0
np.count_nonzero(arr1[:15,0] == 0)


5

In [233]:
jump, v_span, h_span, v_clean, h_clean, RECT_PERC, ARC_PERC, AREA_PERC, BLOCK_PERC = 0, 20, 20, 0.98, 0.98, 0.95, 0.4, 0.45, 0.8

In [234]:
# logic for top left
DIAG = False
coord = []

h, w = img.shape
print(h, w)
for i in range(h):
    if DIAG:
        if i < 290 or i > 296:
            continue
        print('i', i)
        temp_horiz = False

    if h - i < v_span:
        break
    end_vert = i + v_span
    vert_stem = False
    for j in range(w): 
        if w - j < h_span:
            break
        horiz_stem = False
        end_horiz = j + h_span
        #print('end ', end_vert)
        if all(img[i:end_vert,j] == val):
            vert_stem = True    
            if DIAG:
                print('values: ', j , img[i:end_vert,j])
        else:
            if jump > 0:
                ext = end_vert + jump
                if end_vert> h:
                    break
                if np.count_nonzero(img[i:ext, j] == val) >=  v_clean * v_span:
                    #print('here',i,ext,j, v_span, np.count_nonzero(img[i:ext, j] != val))
                    vert_stem = True

        if vert_stem and all(img[i,j:end_horiz] == val):
            horiz_stem = True
        else:
            if jump > 0:
                ext = end_horiz + jump
                if end_horiz > w:
                    break            
                if np.count_nonzero(img[i, j:ext] == val) >= h_clean * h_span:
                    #print('here 2',i,ext,j, h_span, np.count_nonzero(img[i, j:ext] == val))
                    horiz_stem = True
                
        if vert_stem and horiz_stem:
            coord.append([j,i])
            if DIAG:
                temp_horiz = horiz_stem
        vert_stem, horiz_stem = False, False
    if DIAG:
        print(vert_stem, temp_horiz)
            
        

2780 1880


In [235]:
len(coord)
#coord[31000:]

146764

In [236]:
# logic for bottom right
DIAG = False
coord_l = []

h, w = img.shape
print(h, w)
for i in range(h-1,-1, -1):
    if DIAG:
        if i < 290 or i > 296:
            continue
        print('i', i)
        temp_horiz = False

    if i + 1 < v_span:
        continue
    end_vert = i - v_span + 1
    vert_stem = False
    for j in range(w-1, -1, -1): 
        if j + 1 < h_span:
            continue
        horiz_stem = False
        end_horiz = j - h_span + 1
        #print('end ', end_vert)
        if all(img[end_vert:i+1,j] == val):
            vert_stem = True    
            if DIAG:
                print('values: ', j , img[end_vert:i+1,j])
        else:
            if jump > 0:
                ext = end_vert - jump
                if end_vert < 0:
                    continue
                if np.count_nonzero(img[ext:i+1, j] == val) >=  v_clean * v_span:
                    #print('here',i,ext,j, v_span, np.count_nonzero(img[i:ext, j] != val))
                    vert_stem = True

        if vert_stem and all(img[i,end_horiz:j+1] == val):
            horiz_stem = True
        else:
            if jump > 0:
                ext = end_horiz - jump
                if end_horiz < 0:
                    continue            
                if np.count_nonzero(img[i, ext:j+1] == val) >= h_clean * h_span:
                    #print('here 2',i,ext,j, h_span, np.count_nonzero(img[i, j:ext] == val))
                    horiz_stem = True
                
        if vert_stem and horiz_stem:
            coord_l.append([j,i])
            if DIAG:
                temp_horiz = horiz_stem
        vert_stem, horiz_stem = False, False
    if DIAG:
        print(vert_stem, temp_horiz)
            
        

2780 1880


In [237]:
len(coord), len(coord_l)

(146764, 149013)

In [238]:
'''
coord_dict = defaultdict(list)
for k, v in zip(y_coord, x_coord):
    coord_dict[k].append(v)

#coord_dict
len(coord_dict)
'''

'\ncoord_dict = defaultdict(list)\nfor k, v in zip(y_coord, x_coord):\n    coord_dict[k].append(v)\n\n#coord_dict\nlen(coord_dict)\n'

In [239]:
'''
[k for k in coord_dict.keys() if k <= 126]
x_coord[(x_coord) <= 189]
x, y =coord[0]
print(x, y)
[82, 119] in coord    
'''

'\n[k for k in coord_dict.keys() if k <= 126]\nx_coord[(x_coord) <= 189]\nx, y =coord[0]\nprint(x, y)\n[82, 119] in coord    \n'

In [240]:
x = np.array(([3,4],[4,2],[3,3]))
#y = np.transpose(x)
#np.reshape(x, -1), np.reshape(y, -1)
z = np.ravel(x,order='F')
#z = np.flip(np.flip(x, 0), 1)
z = np.flip(z, 0)
w = np.argmax(z)
print(w, z)

ind = np.unravel_index(w, x.shape, 'F')
print(ind)
#np.argmax(x, axis=None)
ymax, ymin = 33, 31
xmax, xmin = 21, 20
y = abs(ymax - ind[0])
x = abs(xmax - ind[1])
y, x


2 [3 2 4 3 4 3]
(2, 0)


(31, 21)

In [241]:
# this is the 2nd of two approaches to trim redundant coordinates. this is very quick.
def trim_t_coords(coord, img, val, polarity):
    DIAG = False
    final = []
    
    h, w = img.shape
    master = np.zeros([h, w])
    lookup = np.zeros([h, w])
    for vals in coord:
        x, y = vals
        master[y, x] = 1
    
    for ix, vals in enumerate(coord):
        if vals == [414, 51]:
            DIAG = True
            print('here')
        else:
            DIAG = False
            
        list_pop = False
        if ix % 1000 == 0:
            print(ix+1, 'out of %d vals %s' %(len(coord),vals) )    
        if DIAG:
            print('vals: ', vals)

        x , y = vals

        if lookup[y, x] <= -1:
            if DIAG:
                print('hmm')
                print(lookup[332, 75], y , x)
            continue
        
        max_x = x + (int(h_span*0.7) - 1) * polarity
        max_y = y + (int(v_span*0.7) - 1) * polarity
        if max_x > w:
            max_x = w * max(0, polarity)
        if max_y > h:
            max_y = h * max(0, polarity)
            
        cut_mat = img[y:max_y+1, x:max_x+1]            
        if np.count_nonzero(cut_mat == val) > BLOCK_PERC * (cut_mat.shape[0] * cut_mat.shape[1]):
            continue
            
        cut_mat = master[y:max_y+1, x:max_x+1]
        if np.count_nonzero(cut_mat) == 1:
            t_x, t_y = x ,y
            #final.append((x, y))
            if DIAG:
                print('1')
        elif [max_x, max_y] in coord:
            t_x ,t_y = max_x, max_y
            list_pop = True
            #final.append((max_x, max_y))
            if DIAG:
                print('2')            
        else:  
            if DIAG:
                print('point 3')  

            list_pop = True
            ravel = np.ravel(cut_mat, order='F')
            flip = np.flip(ravel, 0)
            maxind = np.argmax(flip)
            ind = np.unravel_index(maxind , cut_mat.shape, 'F')
            y_c = max_y - ind[0] * max(-1, polarity)
            x_c = max_x - ind[1] * max(-1, polarity)
            t_x, t_y = x_c, y_c
            #final.append((x_c, y_c))
        
        if list_pop:
            lgth = final.count((x, y))
            [final.remove((x,y)) for i in range(lgth)]
            if DIAG:
                print('oh!')
 
        final.append((t_x, t_y))
        lookup[t_y, t_x] = 5000
        lookup[y:max_y+1, x:max_x+1] += -1
        
        if DIAG:
            print(t_x, t_y)
            print(lookup[332, 75])
            
    return final

In [242]:
# this is the 2nd of two approaches to trim redundant coordinates. this is very quick.
def trim_b_coords(coord, img, val, polarity):
    DIAG = False
    final = []
    
    h, w = img.shape
    master = np.zeros([h, w])
    lookup = np.zeros([h, w])
    for vals in coord:
        x, y = vals
        master[y, x] = 1
    
    for ix, vals in enumerate(coord):
        list_pop = False
        if ix % 1000 == 0:
            print(ix+1, 'out of %d vals %s' %(len(coord),vals) )    
        if DIAG:
            print('vals: ', vals)

        x , y = vals

        if lookup[y, x] <= -1:
            if DIAG:
                print('hmm')
                print(lookup[332, 75], y , x)
            continue
        
        max_x = x + (int(h_span*0.7) - 1) * polarity
        max_y = y + (int(v_span*0.7) - 1) * polarity
        
        if max_x > w or max_x < 0:
            max_x = w * max(0, polarity)
        if max_y > h or max_y < 0:
            max_y = h * max(0, polarity)
            
        cut_mat = img[max_y:y+1, max_x:x+1]            
        if np.count_nonzero(cut_mat == val) > BLOCK_PERC * (cut_mat.shape[0] * cut_mat.shape[1]):
            continue    
            
        cut_mat = master[max_y:y+1, max_x:x+1]
        if np.count_nonzero(cut_mat) == 1:
            t_x, t_y = x ,y
            #final.append((x, y))
            if DIAG:
                print('1')
        elif [max_x, max_y] in coord:
            t_x ,t_y = max_x, max_y
            list_pop = True
            #final.append((max_x, max_y))
            if DIAG:
                print('2')            
        else:  
            if DIAG:
                print('point 3')  

            list_pop = True
            ravel = np.ravel(cut_mat, order='F')
            #flip = np.flip(ravel, 0)
            maxind = np.argmax(ravel)
            ind = np.unravel_index(maxind , cut_mat.shape, 'F')
            y_c = max_y - ind[0] * max(-1, polarity)
            x_c = max_x - ind[1] * max(-1, polarity)
            t_x, t_y = x_c, y_c
            #final.append((x_c, y_c))
        
        if list_pop:
            lgth = final.count((x, y))
            [final.remove((x,y)) for i in range(lgth)]
            if DIAG:
                print('oh!')
 
        final.append((t_x, t_y))
        lookup[t_y, t_x] = 5000
        lookup[max_y:y+1, max_x:x+1] += -1
        
        if DIAG:
            print(t_x, t_y)
            print(lookup[332, 75])
            
    return final

In [243]:
#len(trim_coords(coord))
#w,h 

In [244]:
st = time.time()
#print(len(coord), val)
#coord
final = set(trim_t_coords(coord, thresh.copy(), val,  +1))
final_l = set(trim_b_coords(coord_l, thresh.copy(), val, -1))
print('time: ' , time.time() - st, '\n')

#st = time.time()
#final1 = set(trim1_coords(coord))
#print(len(final))
#print('time: ' , time.time() - st)

1 out of 146764 vals [1, 0]
1001 out of 146764 vals [166, 2358]
2001 out of 146764 vals [101, 2385]
3001 out of 146764 vals [194, 2400]
4001 out of 146764 vals [115, 2410]
5001 out of 146764 vals [177, 2419]
6001 out of 146764 vals [13, 2427]
7001 out of 146764 vals [43, 2434]
8001 out of 146764 vals [172, 2441]
9001 out of 146764 vals [21, 2448]
10001 out of 146764 vals [158, 2453]
11001 out of 146764 vals [14, 2459]
12001 out of 146764 vals [119, 2463]
13001 out of 146764 vals [209, 2467]
14001 out of 146764 vals [81, 2472]
15001 out of 146764 vals [168, 2476]
16001 out of 146764 vals [249, 2480]
17001 out of 146764 vals [222, 2484]
18001 out of 146764 vals [189, 2488]
19001 out of 146764 vals [94, 2492]
20001 out of 146764 vals [34, 2496]
21001 out of 146764 vals [273, 2499]
22001 out of 146764 vals [140, 2503]
23001 out of 146764 vals [71, 2507]
24001 out of 146764 vals [182, 2510]
25001 out of 146764 vals [37, 2514]
26001 out of 146764 vals [230, 2517]
27001 out of 146764 vals [50

78001 out of 149013 vals [181, 2642]
79001 out of 149013 vals [169, 2640]
80001 out of 149013 vals [147, 2638]
81001 out of 149013 vals [98, 2636]
82001 out of 149013 vals [57, 2634]
83001 out of 149013 vals [525, 2631]
84001 out of 149013 vals [469, 2629]
85001 out of 149013 vals [436, 2627]
86001 out of 149013 vals [390, 2625]
87001 out of 149013 vals [298, 2623]
88001 out of 149013 vals [248, 2621]
89001 out of 149013 vals [180, 2619]
90001 out of 149013 vals [123, 2617]
91001 out of 149013 vals [30, 2615]
92001 out of 149013 vals [435, 2612]
93001 out of 149013 vals [307, 2610]
94001 out of 149013 vals [228, 2608]
95001 out of 149013 vals [136, 2606]
96001 out of 149013 vals [46, 2604]
97001 out of 149013 vals [390, 2601]
98001 out of 149013 vals [253, 2599]
99001 out of 149013 vals [140, 2597]
100001 out of 149013 vals [472, 2594]
101001 out of 149013 vals [295, 2592]
102001 out of 149013 vals [176, 2590]
103001 out of 149013 vals [425, 2587]
104001 out of 149013 vals [223, 2585]


In [245]:
set1 = sorted(final.copy(), key=lambda x: (x[0], x[1]))
set2 = sorted(final_l.copy(), key=lambda x: (x[0], x[1]))
len(set1), len(set2)

(78, 92)

In [246]:
set1,set2

([(7, 61),
  (9, 2341),
  (10, 2338),
  (14, 39),
  (32, 28),
  (40, 50),
  (41, 47),
  (49, 9),
  (52, 77),
  (58, 2340),
  (59, 29),
  (63, 8),
  (77, 33),
  (85, 2306),
  (91, 11),
  (102, 2333),
  (120, 19),
  (122, 2323),
  (124, 2306),
  (139, 2301),
  (190, 1537),
  (193, 1662),
  (196, 1787),
  (201, 2354),
  (204, 2368),
  (204, 2369),
  (218, 2410),
  (279, 1555),
  (295, 2465),
  (296, 2457),
  (321, 888),
  (323, 1229),
  (326, 2459),
  (327, 384),
  (331, 1290),
  (367, 2478),
  (370, 1254),
  (376, 2505),
  (391, 1377),
  (393, 889),
  (397, 776),
  (403, 2537),
  (426, 2531),
  (439, 795),
  (448, 1330),
  (448, 1377),
  (457, 2532),
  (465, 888),
  (474, 1327),
  (493, 386),
  (506, 1377),
  (507, 2546),
  (510, 2591),
  (534, 2595),
  (537, 888),
  (563, 2635),
  (564, 1377),
  (579, 864),
  (582, 2640),
  (604, 2669),
  (609, 889),
  (621, 1377),
  (666, 2718),
  (678, 1377),
  (682, 889),
  (736, 1377),
  (754, 889),
  (788, 941),
  (794, 1377),
  (802, 864),
  (826,

In [247]:
x = np.array([[200, 200, 0], [200,0, 200]])
x[np.isin(x, 0)] = -2
y = np.triu(x,0)
cnt_0 = np.count_nonzero(y == 0)
cnt_2 = np.count_nonzero(y == 2)
cnt_tot = y.shape[0] * y.shape[1]
cnt_tot , cnt_2, cnt_0, y
#y[y != 200]

(6, 0, 1, array([[200, 200,  -2],
        [  0,  -2, 200]]))

In [248]:
mil = []

In [249]:
def is_block(img, area, val):
    if np.count_nonzero(img == val) >= area * RECT_PERC:
        #print('cnt ', np.count_nonzero(img == val) , area )
        return True
    else:
        #print('cnt ', np.count_nonzero(img == val) , area )
        return False

def dense_area(surface, val, diaf):
    temp = surface.ravel()
    if diaf:
        print('area perc' , np.count_nonzero(temp == val) , len(temp))
    if np.count_nonzero(temp == val) > len(temp) * AREA_PERC:
        return True
    else:
        return False

# count dark pixels at the 4 corners of the surface
def count_dark_pixels(surface, val, diag, mil3, st):
    #diag = False
    max_y , max_x = surface.shape
    y_len = int(max_y * 0.33)
    x_len = int(max_x * 0.33)
    hit, total = 0, 0
    arc_list = ['top-left', 'top-right', 'bot-left', 'bot-right']
    arc_y = [ (0, y_len) , (0, y_len) , (max_y-y_len, max_y), (max_y-y_len, max_y)] 
    arc_x = [(0, x_len), (max_x - x_len, max_x) , (0, x_len), (max_x - x_len, max_x)]
    #print('surface ', surface)

    for ix, arc in enumerate(arc_list):
        y_axis = arc_y[ix]
        x_axis = arc_x[ix]
        
        a, b, st = shape2_count(surface[y_axis[0]:y_axis[1], x_axis[0]:x_axis[1]], val, arc, y_len, x_len, diag, mil3, st)
        if diag:
            print('a, b' , a ,b)
        hit += a
        total += b
    
    global ONE
    ONE = False
    return (hit, total, st)

        

In [250]:
ONE = True

In [251]:

# find hits (black pixels) from total pixels on surface
def shape2_count(surface, val, shape, i_max, j_max, diag, mil3, st):
    global mil, ONE
    #print('new func')
    #print('surface: ', surface)
    diag = False
    
    surface_c = surface.copy()
    # replace all zeros with -50... this is because np.triu/tril will zero out entries in the matrix    
    surface_c[np.isin(surface_c, 0)] = 50
    if ONE:
        print(surface_c.shape,np.count_nonzero(surface_c == 50))
        #print(surface_c[surface_c != 255])
        #ONE = False
    if val == 0:
        val = 50    
    
    if shape == 'top-left':
        #y_st, y_end, x_st, x_end  = 0, i_max, 0, j_max
        #y_sign, x_sign = 1, 1
        # reverse the matrix        
        surface_c = np.flip(surface_c, axis=1)    
        K = 0
        upper = True
    elif shape == 'top-right':
        #y_st, y_end, x_st, x_end = 0, i_max, j_max-1, -1
        #y_sign, x_sign = 1, -1
        K = 0
        upper = True
        #surface_c = surface_c
    elif shape == 'bot-left':
        #y_st, y_end, x_st, x_end = i_max-1, -1, 0, j_max
        #y_sign, x_sign = -1 , 1
        K = j_max - i_max
        upper = False
        #surface_c = surface.copy()
    else: # bot-right
        # reverse the matrix        
        surface_c = np.flip(surface_c, axis=1) 
        K = j_max - i_max
        upper = False
        #y_st, y_end, x_st, x_end = i_max-1, -1, j_max-1, -1
        #y_sign, x_sign = -1 , -1
    
    mil3.append((time.time()-st))    
    st = time.time()
    
    if upper:
        # zero out entries below a certain diagonal
        surface_new = np.triu(surface_c, k=K)
    else:
        # zero out entries above a certain diagonal
        surface_new = np.tril(surface_c, k=K)
        
    # find all entries that should not be considered
    cnt_reject = np.count_nonzero(surface_new == 0)
    #print(cnt_reject)
    
    hit = np.count_nonzero(surface_new == val)
    total = surface_new.shape[0] * surface_new.shape[1] - cnt_reject
    
    mil.append((time.time() - st))     
    st = time.time()
    return (hit, total, st)

In [252]:
# find hits (black pixels) from total pixels on surface
def shape_count(surface, val, shape, i_max, j_max, diag, mil3, st):
    #print('old func')
    global mil
    #print('surface: ', surface)
    diag = False
    
    if shape == 'top-left':
        y_st, y_end, x_st, x_end  = 0, i_max, 0, j_max
        y_sign, x_sign = 1, 1
    elif shape == 'top-right':
        y_st, y_end, x_st, x_end = 0, i_max, j_max-1, -1
        y_sign, x_sign = 1, -1
    elif shape == 'bot-left':
        y_st, y_end, x_st, x_end = i_max-1, -1, 0, j_max
        y_sign, x_sign = -1 , 1
    else: # bot-right
        y_st, y_end, x_st, x_end = i_max-1, -1, j_max-1, -1
        y_sign, x_sign = -1 , -1
    
    mil3.append((time.time()-st))    
    st = time.time()

    hit, total = 0, 0
    for i in range(y_st, y_end, y_sign):   
        if diag:
            print('i ', i)
        for j in range(x_st, x_end, x_sign):
            if diag:
                print('j ', j)
            if diag:
                print(surface[i, j])
            if surface[i, j] == val:
                hit += 1    
            total += 1
        x_end -= x_sign
        if (x_end - x_st) * x_sign <= -1:
            break

    mil.append((time.time() - st))     
    st = time.time()
    return (hit, total, st)

In [253]:
def pairing(set1, set2, thresh, val):
    global ss_list, mil
    mass_hit, mass_tot = 0, 0
    mil = []
    ss_list = OrderedDict()
    pair_dict = OrderedDict()
    perc_dict = defaultdict(list)
    reject1_list, reject2_list = [], []
    ix = 0
    tot = len(set2)
    global PERC_DICT
    for val1 in set1:
        st = time.time()
        x1 , y1 = val1
        match = False
        if len(set2) == 0:
            break
        #print(val1)
#        mil1.append((time.time() - st ))
        mil1, mil2, mil3, mil4, mil5 = [], [], [], [], []
        for val2 in set2:
            #if val2 != (407, 624):
            #    continue

            hit, total = 0, 0
            x2 , y2 = val2
            if x1 >= x2 or y1 >= y2:
                continue
            #perimeter = 2 * (y2 - y1+1) + 2 * (x2 - x1+1)
            peri_array = []
            dot_array = []
            ratio_array = []
            top_horiz = thresh[y1, x1:x2+1]
            bot_horiz = thresh[y2, x1:x2+1]
            left_vert = thresh[y1:y2+1, x1]
            right_vert = thresh[y1:y2+1, x2]  
            mil1.append((time.time() - st ))
            st = time.time()

            for i in range(3):
                temp_top_horiz = thresh[y1-i, x1-i:x2+1+i]
                temp_bot_horiz = thresh[y2+i, x1-i:x2+1+i]
                temp_left_vert = thresh[y1-i:y2+1+i, x1-i]
                temp_right_vert = thresh[y1-i:y2+1+i, x2+i]
                dot_stack = np.hstack([temp_top_horiz, temp_bot_horiz, temp_left_vert, temp_right_vert])
                cnt = np.count_nonzero(dot_stack == val)  * 1.            
                temp_peri = 2 * (y2+i - (y1-1-i)) + 2 * (x2+i - (x1-1-i))
                ratio_array.append(cnt / temp_peri)
                dot_array.append(cnt)
                peri_array.append(temp_peri)
                #if val1 == (1032, 727) and val2 == (1085, 781):
                #    print(cnt, temp_peri)
            
            mil2.append((time.time() - st ))
            st = time.time()

            if val1 == (445, 212) and val2 == (511, 247):
                print('---------')
                print(val2, ratio_array, dot_array, peri_array)                                               
            
            ratio = max(ratio_array)
            ind = np.argmax(np.array(ratio_array))
            count_dots = dot_array[ind]            
            perimeter = peri_array[ind]
            cut_img = thresh[y1+1:y2 , x1+1:x2]
            # if the two coords are connected by rectangle then process...
            ss_list[(val1,val2)]= float(count_dots) / perimeter            
            #if val1 == (1594, 793):
            #    print('xxxx ', val2, count_dots, perimeter)     
            #print(val2, ratio)
            diaf = False
            if val1 == (177, 338) and val2 == (214, 356):
                    print('---------')
                    diaf = True            
            
            max_empty_pixels = (1 - RECT_PERC) * 100
            forward = False

            if perimeter >= 100:
                diff = perimeter - count_dots
                if diff < max_empty_pixels:
                    forward = True
            else:
                if ratio >= RECT_PERC:
                    forward = True
            if diaf:
                print('max_empty_pixels ', max_empty_pixels, perimeter, diff )
            #if ratio >= RECT_PERC:
            if forward:
                area = len(left_vert-2) * len(top_horiz-2)
                # if these points are in dark block then ignore them: it would be a logo or fat margin; or else, proceed...
                    #print(val2, ratio_array, dot_array, peri_array)                   
                if is_block(cut_img, area, val):
                    pass                
                else:
                    if val1 == (192, 87):                    
                        diag = True
                    else:
                        diag = False
                    #mil3.append((time.time() - st ))
                    #st = time.time()  
                    #print('cut ', cut_img.shape)
                    # proceed only if its a rectangular connection between the coords.. if it's curvy, then most likely
                    # it would be some character.. for curves, triangular arcs at 4 edges will be dark
                    hit, total, st = count_dark_pixels(cut_img, val, False, mil3, st)
                    #if val2 == (407, 624):
                    if diaf:
                        print(hit, total)
                      #print(hit, total)
                    mass_hit += hit
                    mass_tot += total
                    mil4.append((time.time() - st ))
                    st = time.time()                    
                    if hit < total * ARC_PERC:
                        # if rectangular space is dense with dark pixels then it's not a rectangle with a character
                        if dense_area(cut_img, val , diaf):
                            if diaf:
                                print('oh!!)')
                            #pass
                        else:
                            if diaf:
                                print('yes!!')
                            match = True
                            pair_dict[val1] = val2
                            temp_perc = float(count_dots) / perimeter
                            perc_dict[val1].append((val2, temp_perc))
                            #print(val1, val2, count_dots , perimeter)  
                    else:
                        pass
                        #print(val1, val2, perimeter, count_dots)
                        #print('Dark ARC... ', hit, total)     
                #break
                    #else:
                    #    print('no ARC... ', hit, total)  
            mil5.append((time.time() - st ))
            st = time.time()      
            #if val2 == (407, 624):
            #    break
            

        if val1 == (2450,2827):
            print('mark..', val2, count_dots, perimeter, hit, total, match)
        if match:
            #pass
            #set2.remove(val2)
            match = False
        else:
            reject1_list.append(val1)
        
        ix += 1
        if ix % 100 == 0:
            print('---------- %d out of %d processed.. ' %(ix, tot) )
        #print('Sum : ', sum(mil1), sum(mil2), sum(mil3), sum(mil4), sum(mil5), sum(mil), sum(mil1+mil2+mil3+mil4+mil5))
        
        if ix >= 200000 :
             break
    
    #reject2_list.extend(set2)
    print(mass_hit, mass_tot)
    
    print('reject1 list ' , len(reject1_list), reject1_list,'\n')
    #print(len(reject2_list), reject2_list,'\n')
    print(perc_dict, '\n')
    
    print('pair_dict length ',len(pair_dict), '\n')
    #PERC_DICT = perc_dict
    return perc_dict
    #return pair_dict

        

In [254]:
# pairing function is not perfect when a start coord has multiple end coords... below function tries to bring a consensus
def consensus(perc_dict):
    global TEMP
    filter_list = []
    #lst = {'a':('s21',0.90), 'b':('s21', 0.98), 'c':('s20', 0.96), 'd':('s21', 0.94)}
    lst = perc_dict
    #print(lst.items())
    # get val2 & %... sort by %, val2 descending... if there are duplicates val2, one with higest % will come at top
   
    # sort val1 in descending order of y axis and x axis
    temp_perc_dict = OrderedDict(sorted(perc_dict.items(), key=lambda x: (x[0][1] + x[0][0]) , reverse=True))
    
    val2_list = []    
    # extract only val2 as a reference dataset
    for entry in lst.values():
        for val, perc in entry:
            val2_list.append(val)
    
    # remove duplicates    
    val2_set = set(val2_list)
    #print(val2_list)
    
    # iterate thru val1... if val2 is in above reference set, then save and pop from above set so that next val1 with same \
    # val2 does not find val2.. so a val2 will be assigned to only 1 val1.. a val1 will get val2 only if it has highest %  \
    # compared to other val1
    for k, v in temp_perc_dict.items():
        temp_v = v
        # if there are multiple val2 for a val1 key, then sort the val2 in descending order of %
        temp_v = sorted(temp_v, key= lambda x: x[1], reverse=True) 
        # sort val2 in order of y axis + then x axis        
        temp_v = sorted(temp_v, key=lambda x: (x[0][1] + x[0][0]))
        
        #print(v)
        #val2, perc = temp_v[0]
        if k == (445, 212):
            print('\n*******************\n')
            print(temp_v)
            print('\n*******************')
        atleast_1 = False        
        for val2, perc in temp_v:
            if val2 in val2_set:
                #print(k, v)
                filter_list.append((k, val2, perc))
                val2_set.remove(val2)
                atleast_1 = True
                break
            else:
                pass
        if not atleast_1:
            print('skip ', k, temp_v)
    
    return filter_list

#x = consensus(inter_dict)


In [255]:
'''
z = np.array(list(ss_list.values()))
x = np.max(z)
y = z[np.where(z >= 0.9)]
x = y[np.where( y <= 1.0 )]
for k, v in ss_list.items():
    if v in x:
        print(k, v)
        #pass
#y
'''
print(np.__version__)

1.14.1


In [256]:
#coord_l
st = time.time()
inter_dict = pairing(set1.copy(), set2.copy(), thresh, val)
print('\n',time.time()-st)
final_list = consensus(inter_dict)
#print(final_list)

#final_dict = pairing(set1.copy(), set2.copy(), thresh, val)
#print(final_dict)
#101773140 388878072
#101773140 388878072


(6, 5) 30
(6, 5) 19
(6, 5) 14
(6, 5) 29
52518 406076
reject1 list  55 [(7, 61), (9, 2341), (10, 2338), (14, 39), (32, 28), (40, 50), (41, 47), (49, 9), (52, 77), (58, 2340), (59, 29), (63, 8), (77, 33), (85, 2306), (91, 11), (102, 2333), (120, 19), (122, 2323), (124, 2306), (139, 2301), (201, 2354), (204, 2368), (204, 2369), (218, 2410), (279, 1555), (295, 2465), (296, 2457), (323, 1229), (326, 2459), (327, 384), (331, 1290), (367, 2478), (370, 1254), (376, 2505), (397, 776), (403, 2537), (426, 2531), (439, 795), (448, 1330), (457, 2532), (474, 1327), (493, 386), (507, 2546), (510, 2591), (534, 2595), (563, 2635), (579, 864), (582, 2640), (604, 2669), (666, 2718), (788, 941), (802, 864), (848, 931), (891, 1103), (1206, 507)] 

defaultdict(<class 'list'>, {(190, 1537): [((278, 1583), 1.0)], (193, 1662): [((281, 1707), 1.0)], (196, 1787): [((284, 1833), 1.0)], (321, 888): [((393, 968), 0.9902597402597403), ((465, 968), 0.9933628318584071), ((682, 968), 0.9966139954853274), ((754, 968), 0

In [257]:
print(len(final), sorted(final),'\n')
print(len(final_l), sorted(final_l), '\n')
print('after pairing... ' , len(inter_dict))

78 [(7, 61), (9, 2341), (10, 2338), (14, 39), (32, 28), (40, 50), (41, 47), (49, 9), (52, 77), (58, 2340), (59, 29), (63, 8), (77, 33), (85, 2306), (91, 11), (102, 2333), (120, 19), (122, 2323), (124, 2306), (139, 2301), (190, 1537), (193, 1662), (196, 1787), (201, 2354), (204, 2368), (204, 2369), (218, 2410), (279, 1555), (295, 2465), (296, 2457), (321, 888), (323, 1229), (326, 2459), (327, 384), (331, 1290), (367, 2478), (370, 1254), (376, 2505), (391, 1377), (393, 889), (397, 776), (403, 2537), (426, 2531), (439, 795), (448, 1330), (448, 1377), (457, 2532), (465, 888), (474, 1327), (493, 386), (506, 1377), (507, 2546), (510, 2591), (534, 2595), (537, 888), (563, 2635), (564, 1377), (579, 864), (582, 2640), (604, 2669), (609, 889), (621, 1377), (666, 2718), (678, 1377), (682, 889), (736, 1377), (754, 889), (788, 941), (794, 1377), (802, 864), (826, 889), (848, 931), (851, 1377), (891, 1103), (898, 889), (909, 1377), (970, 888), (1206, 507)] 

92 [(26, 2334), (31, 61), (39, 58), (48, 

In [258]:
#start = final_dict.keys()
start = list(map(lambda x: x[0], final_list))
#end = final_dict.values()
end = list(map(lambda x: x[1], final_list))
print(len(start), len(end))
#start, end

22 22


In [259]:
paint = [7,7]
for (x, y) in start:
    cv2.rectangle(img2,(x,y), (x+paint[0],  y+paint[1]), (110,110,110),2)
    #cv2.rectangle(img2,(x,y), (x+h_span, y+v_span), (110,110,110),3)
cv2.imwrite(''.join(('final_', filename, '.png')), img2)

True

In [260]:
paint = [7,7]
for (x, y) in end:
    cv2.rectangle(img2,(x,y), (x-paint[0],  y-paint[1]), (110,110,110),2)
    #cv2.rectangle(img2,(x,y), (x+h_span, y+v_span), (110,110,110),3)
cv2.imwrite(''.join(('final_', filename, '.png')), img2)

True

In [261]:
temp_sort = sorted(final_list, key=lambda x: (x[0], x[1], x[2]) )
temp_list = list(map(lambda x: ''.join((str(x[0]) , ';' , str(x[1]) )) , temp_sort ))
#for y_val, x_val in temp_list:
str1 = "\n".join(temp_list)
with open(''.join(('csv_', filename, '.csv')), 'w', newline='\n') as fo:
    writer = csv.writer(fo, delimiter=',')
    writer.writerow(('Start_coord', 'End_coord'))
    writer.writerow(('(x , y)', '(x , y)'))
    for row in temp_list:
        c1, c2 = row.split(';')
        writer.writerow((c1,c2))

In [262]:
#temp_list[:4]
#temp_sort[:4]
#str1

In [263]:
fail

NameError: name 'fail' is not defined

In [ ]:
merg=zip(start,end)
crop=[]
for k,v in merg:
    x1,y1=k
    x2,y2=v
    #print(x1,y1,x2,y2)
    
    w=abs(x2-x1)
    h=abs(y2-y1)
    #print('x=',x1,'y=',y1,'w=',abs(x2-x1),'h=',abs(y2-y1),'\n')
    crop.append(img[y1+2:y1+h-2,x1+2:x1+w-2])  

In [ ]:
k=0
for i in crop:
    #len(crop)
    
    #cv2.imwrite("sample_v_"+str(k)+'.jpg',i)
    cv2.imshow('imag',i)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()
    k+=1

In [ ]:
fail

In [ ]:
cut = thresh[211:249, 444:513].copy()
#cut = thresh[140:152, 51:275].copy()
#cut = thresh[3259:3295, 701:736].copy()
#cut = thresh[4177:4233, 2005:2067].copy()
cv2.imshow('cut', cut)
cv2.waitKey(10000)
cv2.destroyAllWindows()


In [ ]:
#x = cut.ravel()
tmp = cut[0,:]
tmp = cut[:,57]
np.count_nonzero(tmp == 0), len(tmp)
#tmp, cut.shape
#np.count_nonzero(x == 255),  np.count_nonzero(x == 0), len(x)